In [41]:
import os, sys
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

### Loading Data

In [42]:
# Load data configuration
data_dir = r'C:\Users\alail\OneDrive\Documentos\dado_iniciais'
# data_dir = r'D:\Pre-Processing Data'
pen = [10, 50, 75]

columns_name = [
    'File ID',
    'Fault type',
    'Fault resistance', 
    'Incidence angle',
    'Fault location',
    'Fault distance',
]
df_cases = pd.read_csv(
    os.path.join(data_dir, 'cenarios_falta_preprocessamento.csv'),
    header=None,
    index_col=False,
    names=columns_name
)


In [43]:
df_cases = df_cases[(df_cases['Fault type'] == 1)|(df_cases['Fault type'] == 4)|(df_cases['Fault type'] == 8)|(df_cases['Fault type'] == 11)]
print(len(df_cases))

540


In [44]:
# Load data
if os.path.exists(os.path.join(data_dir, 'data.npy')) and False:
    data = np.load(os.path.join(data_dir, 'data.npy'))
else:
    one_data_size = (6, 3072)
    print(len(df_cases))
    data = np.zeros((len(pen) * len(df_cases), *one_data_size))
    for i, p in enumerate(pen):
        n = 0
        for j, case in df_cases.iterrows():
            data[i * len(df_cases) + n] = np.loadtxt(
                os.path.join(data_dir, f'pen{p}', f'sinal_{j+1}_.csv'),
                delimiter=','
            ).T
            n += 1
    np.save(os.path.join(data_dir, 'data.npy'), data) # save in numpy for faster loading

540


In [45]:
# Create a Dataframe to store the features
columns_name = [
    'File ID',
    'Fault type',
    'Fault resistance', 
    'Incidence angle',
    'Fault location',
    'Fault distance',
    'Pen',
]
columns_type = {
    'File ID': int,
    'Fault type': int,
    'Fault resistance': float,
    'Incidence angle': float,
    'Fault location': int,
    'Fault distance': float,
    'Pen': int
}
df_features = pd.DataFrame(
    columns= columns_name,
    index=range(len(pen) * len(df_cases))
    
)
for i, p in enumerate(pen):
    n = 0
    for j, case in df_cases.iterrows():
        df_features.loc[i * len(df_cases) + n] = case.tolist() + [p]
        n += 1
df_features = df_features.astype(columns_type)
del df_cases

In [46]:
df_features = df_features[(df_features['Fault type'] == 1)|(df_features['Fault type'] == 4)|(df_features['Fault type'] == 8)|(df_features['Fault type'] == 11)]

In [47]:
def save_dataframe_image(df, filename, n = 8):
    import dataframe_image as dfi
    first = df.head(n)
    last = df.tail(n)
    placeholder = pd.DataFrame({col: ['...'] for col in df.columns})
    placeholder = placeholder.rename(index={0: '...'})
    combined_df = pd.concat([first, placeholder, last], ignore_index=False)
    combined_df.columns.rename('Index', inplace=True)
    dfi.export(combined_df, filename, dpi=300, max_cols=10)
save_dataframe_image(df_features, r'features_df.png', n=4)

### Fourier Features

In [48]:
from Features_Fourier import dft_features
idx_left = data.shape[-1]//2 - 256
idx_right = data.shape[-1]//2 + 512*2
df_fourier = df_features.copy()
fs = 512*60
for i in tqdm(range(data.shape[0])):
    feat = dft_features(np.vstack([data[i,3,idx_left:idx_right], data[i,0,idx_left:idx_right]]), fs)
    for j, f in enumerate(feat):
        df_fourier.loc[i, f'Feature {j}'] = f 
df_fourier.to_csv(
    r'features/fourier_features.csv', 
    index=True)
save_dataframe_image(df_fourier, r'features/fourier_features.png')

  0%|          | 0/1620 [00:00<?, ?it/s]

### Wavelet Features

In [49]:
from MetricasWavelet import metricas_yordanos
df_wavelet = df_features.copy()
for i in tqdm(range(data.shape[0])):
    feat = metricas_yordanos(data[i,3,:].T)
    for j, f in enumerate(feat):
        df_wavelet.loc[i, f'Feature {j}'] = f
df_wavelet.to_csv(
    r'features/wavelet_features.csv', 
    index=True
)
save_dataframe_image(df_wavelet, 'features/wavelet_features.png')

  0%|          | 0/1620 [00:00<?, ?it/s]

### Stockwell Features

In [50]:
from Features_Stockwell import extract_feature_TS
df_stockwell = df_features.copy()
for i in tqdm(range(data.shape[0])):
    feat = np.concatenate([
        extract_feature_TS(data[i,3,:], tempo_inicio = data.shape[-1]/fs/2, sample_cycle=512)
    ])
    for j, f in enumerate(feat):
        df_stockwell.loc[i, f'Feature {j}'] = f
df_stockwell.to_csv(
    r'features/stockwell_features.csv', 
    index=True
)
save_dataframe_image(df_stockwell, 'features/stockwell_features.png')


  0%|          | 0/1620 [00:00<?, ?it/s]

### Three-Phase Features

In [51]:
from Features_TT import extract_feature_TT
df_tt = df_features.copy()
for i in tqdm(range(data.shape[0])):
    feat = extract_feature_TT(
        signal_voltage_A = data[i,0,:],
        signal_voltage_B = data[i,1,:],
        signal_voltage_C = data[i,2,:],
        signal_corrent_A = data[i,3,:],
        signal_corrent_B = data[i,4,:],
        signal_corrent_C = data[i,5,:],
        tempo_inicio = data.shape[-1]/fs/2,
        sample_cycle = 512
    )
    for j, f in enumerate(feat):
        df_tt.loc[i, f'Feature {j}'] = f
df_tt.to_csv(r'features/tt_features.csv', index=True) 
save_dataframe_image(df_tt, 'features/tt_features.png')

  0%|          | 0/1620 [00:00<?, ?it/s]

### EMD Features

In [52]:
# from Metricas_EMD import apply_vmd, svd
from scipy.signal import hilbert
from pyemd.EMD import EMD
df_emd = df_features.copy()
fault_pos = data.shape[-1]//2
for i in tqdm(range(data.shape[0])):
    feat = np.zeros((3,3))
    for idx in range(1):
        emd = EMD.EMD()
        imfs = emd(data[i,idx,fault_pos-64:fault_pos+63], max_imf=10)
        feat[idx, :] = np.array([
            np.sum(imfs[:,0]**2),
            np.std(np.abs(imfs[:,0])),
            np.std(np.angle(hilbert(imfs[:,0]))),
        ])
    feat = feat.reshape(-1)
    for j, f in enumerate(feat):
        df_emd.loc[i, f'Feature {j}'] = f
df_emd.to_csv(r'features/EMD_features.csv', index=True) 
save_dataframe_image(df_emd, 'features/EMD_features.png')

ModuleNotFoundError: No module named 'pyemd'

### VMD Features

In [ ]:
from Metricas_VMD import apply_vmd, svd
df_vmd = df_features.copy()
fault_pos = data.shape[-1]//2
for i in tqdm(range(data.shape[0])):
    svd(apply_vmd(data[i,0,fault_pos:fault_pos+22]), full_matrices=False)[1],
    for j, f in enumerate(feat):
        df_vmd.loc[i, f'Feature {j}'] = f
df_vmd.to_csv(r'features/VMD_features.csv', index=True) 


  0%|          | 0/4050 [00:00<?, ?it/s]

In [ ]:
save_dataframe_image(df_vmd, 'features/VMD_features.png')

In [ ]:
df_fourier = pd.read_csv(r'features/fourier_features.csv', index_col=0)
df_wavelet = pd.read_csv(r'features/wavelet_features.csv', index_col=0)
df_stockwell = pd.read_csv(r'features/stockwell_features.csv', index_col=0)
df_tt = pd.read_csv(r'features/tt_features.csv', index_col=0)
df_emd = pd.read_csv(r'features/EMD_features.csv', index_col=0)
df_vmd = pd.read_csv(r'features/VMD_features.csv', index_col=0)

array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)